<a href="https://colab.research.google.com/github/prvshkmr/prvshkmr.github.io/blob/main/Stroke_Predication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stroke Prediction**

In [ ]:
# Connecting to drive

from google.colab import drive
drive.mount('/content/drive')

folder_path = '/content/drive/My Drive/PRML_Project/'
data_path = folder_path + 'healthcare-dataset-stroke-data.csv'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Required Modules
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#Read the data into a pandas DataFrame
df = pd.read_csv(data_path)

#Print the first few rows of the DataFrame
print(df.head())


      id  gender   age  hypertension  heart_disease ever_married  \
0   9046    Male  67.0             0              1          Yes   
1  51676  Female  61.0             0              0          Yes   
2  31112    Male  80.0             0              1          Yes   
3  60182  Female  49.0             0              0          Yes   
4   1665  Female  79.0             1              0          Yes   

       work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0        Private          Urban             228.69  36.6  formerly smoked   
1  Self-employed          Rural             202.21   NaN     never smoked   
2        Private          Rural             105.92  32.5     never smoked   
3        Private          Urban             171.23  34.4           smokes   
4  Self-employed          Rural             174.12  24.0     never smoked   

   stroke  
0       1  
1       1  
2       1  
3       1  
4       1  


In [ ]:
print(df.isnull().sum())

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64


In [ ]:
# Count the occurrences of 'Unknown' in the 'smoking_status' column
unknown_count = (df['smoking_status'] == 'Unknown').sum()

# Print the count of 'Unknown' values
print("Number of 'Unknown' values in smoking_status column:", unknown_count)

Number of 'Unknown' values in smoking_status column: 1544


In [ ]:
# Get the number of data points (patients) and features
num_data_points, num_features = df.shape

# Print the number of data points and features
print("Number of data points (patients):", num_data_points)
print("Number of features:", num_features)

Number of data points (patients): 5110
Number of features: 12


In [ ]:
missing_values = df.isnull().sum()

# Calculate percentage of missing values in each column
missing_percentage = (missing_values / len(df)) * 100

# Print missing values and their percentage
print("Missing Values:")
print(missing_values)
print("\nPercentage of Missing Values:")
print(missing_percentage)

Missing Values:
id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

Percentage of Missing Values:
id                   0.000000
gender               0.000000
age                  0.000000
hypertension         0.000000
heart_disease        0.000000
ever_married         0.000000
work_type            0.000000
Residence_type       0.000000
avg_glucose_level    0.000000
bmi                  3.933464
smoking_status       0.000000
stroke               0.000000
dtype: float64


# **K-Nearest Neighbors (KNN)**

# **Support Vector Machines (SVM)**

# **Decision Tree**

# **Principal Component Analysis (PCA)**

# **Logistic Regression**